# GPN Cup 2024. Тематическое моделирование
# Трапер Максим

## Задача
В архиве вы найдете файл, который предварительно оцифровал и обработал Лютик

1. Предложите решение по выделению подмножества тематик и их описания из общего корпуса цитат.
    
2. Предположим, Йеннифер и Трисс поставят эту активность как регулярную, проводя интервью раз в месяц по разным доменам предприятия. Предложите варианты, как Лютику сделать для них легковесную систему по обработке новых цитат из опросов с учетом уже накопленной истории извлеченных тематик и тегов.



# 0. Теоретическое введение

# Push notebook to Git (for Google Colaboratory)

# 1. Ход решения

## 1.1 Предварительная подготовка

In [ ]:
! git clone https://github.com/MaksimTraper/TopicModel_GPN_Cup_2024.git

### 1.1 Установка библиотек

In [ ]:
!pip install langchain-community langchain-core
!pip install pyspellchecker
!pip install llama-cpp-python

! git clone https://github.com/ai-forever/sage.git

%cd sage
! pip install .
! pip install -e .[errant]

!pip install pyaspeller

fatal: destination path 'sage' already exists and is not an empty directory.
/content/sage
Processing /content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
augmentex is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Created wheel for sage: filename=sage-1.1.0-py3-none-any.whl size=47141 sha256=f7beee45d00f156222cf73d253a09e5944fd8ae8e6417a6ca06b473462dc654c
  Stored in directory: /tmp/pip-ephem-wheel-cache-y_98galb/wheels/f6/fb/b3/1a788356f44a49d04bc8fdc67921c8b7f36cef909ee0b9615e
Successfully built sage
  Attempting uninstall: sage
    Found existing installation: sage 1.1.0
    Uninstalling sage-1.1.0:
      Successfully uninstalled sage-1.1.0
Obtaining file:///content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.

In [ ]:
import json
import re
from collections import Counter

from nltk.corpus import stopwords
import nltk
from nltk.util import ngrams
from nltk.metrics import edit_distance
from spellchecker import SpellChecker

from langchain import HuggingFaceHub, LLMChain, PromptTemplate

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM,LogitsProcessor
from llama_cpp import Llama

import os
import torch
from sage.spelling_correction import T5ModelForSpellingCorruption, RuM2M100ModelForSpellingCorrection, AvailableCorrectors

nltk.download('stopwords')
# Загрузка стоп-слов для русского языка
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('russian'))

In [ ]:
stop_words.remove('не')

### 1.2 Загрузка исходных файлов

In [ ]:
%cd ..

/content


In [ ]:
file_path_quotes = 'cintra_phoenix_oils_hr_mgck_feather.json'

with open(file_path_quotes, 'r', encoding='utf-8') as f:
  data = json.load(f)

quotes = []
for quote in data:
  quotes.append(quote['quote'])

In [ ]:
file_path_abbrev = 'abbreviations.txt'
abbrev_text = open(file_path_abbrev).read().split('\n')

abbreviations = {}
for abb in abbrev_text[:len(abbrev_text)-1]:
  abbrev = abb.split(' - ')
  abbreviations[abbrev[0].lower()] = abbrev[1].lower()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/RuM2M100-418M")
corrector = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/RuM2M100-418M")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
corrector.model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

M2M100Model(
  (shared): M2M100ScaledWordEmbedding(14341, 1024, padding_idx=1)
  (encoder): M2M100Encoder(
    (embed_tokens): M2M100ScaledWordEmbedding(14341, 1024, padding_idx=1)
    (embed_positions): M2M100SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-11): 12 x M2M100EncoderLayer(
        (self_attn): M2M100Attention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): ReLU()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
 

In [ ]:
protected_words = ['плантир']
protected_ids = [tokenizer.convert_tokens_to_ids(word) for word in protected_words]

In [ ]:
# Собственный LogitsProcessor для защиты токенов
class ForceTokensLogitsProcessor(LogitsProcessor):
    def __init__(self, protected_token_ids, penalty=-1000.0):
        super().__init__()
        self.protected_token_ids = protected_token_ids
        self.penalty = penalty  # сильный штраф для защищённых слов

    def __call__(self, input_ids, scores):
        # Применяем штраф к защищённым токенам
        for token_id in self.protected_token_ids:
            scores[:, token_id] += self.penalty
        return scores

def generate_fixed_from_samples(model, tokenizer, samples, device='cpu'):
    model.eval()
    model = model.to(device)

    tokens = tokenizer(samples, padding=True, return_tensors='pt')
    output = model.generate(tokens['input_ids'].to(device),
                            do_sample=True, top_k=50, top_p=0.95,
                            num_return_sequences=1,
                            logits_processor=[ForceTokensLogitsProcessor(protected_ids, -1000)])
    results = tokenizer.batch_decode(output.cpu(), skip_special_tokens=True)

    return dict(zip(samples, results))

In [ ]:
tokenizer = tokenizer
model = corrector

samples = list(results.values())
results = dict()

fixed_samples = generate_fixed_from_samples(model, tokenizer, samples, device=device)
results = fixed_samples

AttributeError: 'T5ModelForSpellingCorruption' object has no attribute 'eval'

In [ ]:
for original, result in zip(list(results.keys()), list(results.values())):
    print(f"'{original}'")
    print(f"{result}'")
    print('-'*50)

In [ ]:
corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_fredt5_distilled_95m.value)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
corrector.model.to(device)

results = dict()
samples = quotes[0:20]
model = corrector.model
tokenizer = corrector.tokenizer

fixed_samples = generate_fixed_from_samples(model, tokenizer, samples, device=device)
results = fixed_samples

for original, result in zip(list(results.keys()), list(results.values())):
    print(f"'{original}'")
    print(f"{result}'")
    print('-'*50)

'«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» 


'
"У нас среди ночи в районе 55 часов упала полxа с водой в стеклянной таре, то есть посреди этой лужа стеккла. Всё, это уже под утро (кстати, это не первый раз). Они сами по себе падают, как-то неправильно рассчитывают, мы же должны выставлять по определённой картинке. У нас при мне было уже пару раз, что-то сами по себе грохаются эти полки с бутылками".'
--------------------------------------------------
'Программы повышения квалификации через гильдию ЦМФ, хитрые, хитрые.  У всех, кто хочет повышаться, есть свои предпочтения и направления, а гильдия... гильдия толкает свое, не всегда то, что нужно сотрудникам.  В "Белых 

### 1.3 Предобработка текстов

In [ ]:
def find_common_bigrams(text_list):
    bigram_counts = Counter()
    for text in text_list:
        tokens = text.split()
        bigrams = ngrams(tokens, 1)
        bigram_counts.update(bigrams)
    return bigram_counts.most_common()

bigram_counts = find_common_bigrams(quotes)
print(bigram_counts)

In [ ]:
# Инициализация для русского языка
spell = SpellChecker(language='ru')

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление повторяющихся символов (например, "оооо" -> "о")
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Удаление знаков препинания (если не нужны)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Удаление стоп-слов
    words = text.split()
    words = [abbreviations[word] if word in abbreviations.keys() else word for word in words]
    text = ' '.join([word for word in words if word not in stop_words])

    #words = text.split()
    #corrected_words = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
    #return ' '.join(corrected_words)

    return text

preprocessed_quotes = [preprocess_text(quote) for quote in quotes]

In [ ]:
print(quotes[1])
print(preprocessed_quotes[1])

Программы повышения квалификации через гильдию ЦМФ, хитрые, хитрые.  У всех, кто хочет повышаться, есть свои предпочтения и направления, а гильдия... гильдия толкает свое, не всегда то, что нужно сотрудникам.  В "Белых Змеях", например,  программы гибче,  и сотрудники сами выбирают, что им нужно. 

програмы повышения квалификаци гильдию цмф хитрые хитрые хочет повышаться свои предпочтения направления гильдия гильдия толкает свое не нужно сотрудникам белых змеях например програмы гибче сотрудники сами выбирают нужно


In [ ]:
# Шаг 1: Собираем уникальные слова из всех текстов
unique_words = set(word for quote in preprocessed_quotes[:1] for word in quote.split())

# Шаг 2: Формируем словарь предполагаемых исправлений
correction_dict = {}
for word in unique_words:
    if word not in spell:  # Если слово считается ошибочным
        correction_dict[word] = spell.correction(word)

# Шаг 3: Печатаем словарь для ручной проверки и корректировки
print("Словарь предложенных исправлений:")
for original, correction in correction_dict.items():
    print(f"{original}: {correction}")

"""
# Ручная корректировка (пример)
correction_dict["класный"] = "классный"
correction_dict["Привеетт"] = "Привет"

# Шаг 4: Применение исправлений к тексту
def correct_text_with_dict(text, correction_dict):
    words = text.split()
    corrected_words = [correction_dict.get(word, word) for word in words]
    return ' '.join(corrected_words)

# Пример использования
text = "Привеетт всем, это класный день"
corrected_text = correct_text_with_dict(text, correction_dict)
print("Исправленный текст:", corrected_text)
"""

Словарь предложенных исправлений:
грохаются: None
полxа: пола
таре: тара
«у: у
стекляной: None
както: като
лужа: луна
5: 5
ночi: ночь
бутылками»: None
картинке: картинка
выставлять: заставлять
первыи: первый
расчитывают: рассчитываю
определёной: определённо


'\n# Ручная корректировка (пример)\ncorrection_dict["класный"] = "классный"\ncorrection_dict["Привеетт"] = "Привет"\n\n# Шаг 4: Применение исправлений к тексту\ndef correct_text_with_dict(text, correction_dict):\n    words = text.split()\n    corrected_words = [correction_dict.get(word, word) for word in words]\n    return \' \'.join(corrected_words)\n\n# Пример использования\ntext = "Привеетт всем, это класный день"\ncorrected_text = correct_text_with_dict(text, correction_dict)\nprint("Исправленный текст:", corrected_text)\n'

In [ ]:
edit_distance('цмф', 'мы')

2

In [ ]:
word='Привеетт'

correction = spell.correction(word)

if spell.known([correction]) and edit_distance(word, correction) < 2:
    print(correction)
else:
    print(word)

привет


In [ ]:
# Инициализация для русского языка
spell = SpellChecker(language='ru')

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление повторяющихся символов (например, "оооо" -> "о")
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Удаление знаков препинания (если не нужны)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Удаление стоп-слов
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])

    words = text.split()
    corrected_words = [spell.correction(word) if word not in spell else word for word in words]
    return ' '.join(corrected_words)

    return text

# Пример текста
text = "Привеетт!! Как дела???😊😊😊 Как твоя раб0тта в ЦМФ?"

# Предобработка текста
preprocessed_text = preprocess_text(text)
print("Предобработанный текст:", preprocessed_text)


Предобработанный текст: привет делай твоя работа мы


## 2.1 Основная часть

## 2.1.1 Генерация тегов

In [ ]:
llm = Llama.from_pretrained(
	repo_id="RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4_gguf",
	filename="FP16.gguf",
  use_gpu=True,
  device_map="auto",
  n_ctx=2048
)

In [ ]:
import ctypes

# Получение указателя на функцию mlock
libc = ctypes.CDLL("libc.so.6")  # Для Linux; на macOS используйте "libSystem.dylib"
mlock = libc.mlock

# Блокировка памяти для предотвращения выгрузки
def lock_memory(address, length):
    result = mlock(address, length)
    if result != 0:
        raise RuntimeError("Ошибка при вызове mlock для блокировки памяти")

# Получение размера памяти модели (например, 1 ГБ)
model_memory_size = 1 * 1024 * 1024 * 1024 * 12  # 1 ГБ
address = ctypes.c_void_p(id(llm))  # Получаем адрес объекта модели

lock_memory(address, model_memory_size)


In [ ]:
unique_tags = []
for text in quotes[0:10]:
  print(text)
  prompt = (
      f"Далее будет дан текст. Для него нужно сгенерировать от 5 до 10 \
      тегов (длиной от одного до трёх слов), точно описывающих то, что \
      происходит в тексте и о чём или о ком говориться в нём. Если в тексте есть имена собственные или аббревиатуры, обязательно добавляй их в теги.\
      Пример, дан текст: Мурр... Новиград - город контрастов, да. Но в ЦМФ, как и в других ЗМС, люди превратились в котов, все заняты только собой. Никто не хочет протянуть лапку товарищу, помочь с грузом. Все в напряжении, в спешке. Мурр, раньше было лучше, когда работали втроем, а не порознь.\
      Теги: Новиград, ЦМФ, ЗМС, эгоизм, раньше было лучше, Работа"
      f"Текст: {text}; \n Теги:"
  )

  tags = generate_tags(prompt, llm)
  unique_tags.extend(tags)
  print("Generated tags:", tags)

# Удаление дубликатов
unique_tags = list(set(unique_tags))

«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» 





Llama.generate: 3 prefix-match hit, remaining 311 prompt tokens to eval
llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   311 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   57022.62 ms /   336 tokens
Llama.generate: 186 prefix-match hit, remaining 88 prompt tokens to eval


Generated tags: ['Уличные инциденты', 'полxа с водой', 'водно-опасные предметы', 'расчётные ошибки', 'безопасность в районе']
Программы повышения квалификации через гильдию ЦМФ, хитрые, хитрые.  У всех, кто хочет повышаться, есть свои предпочтения и направления, а гильдия... гильдия толкает свое, не всегда то, что нужно сотрудникам.  В "Белых Змеях", например,  программы гибче,  и сотрудники сами выбирают, что им нужно. 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    88 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20571.85 ms /   103 tokens
Llama.generate: 186 prefix-match hit, remaining 93 prompt tokens to eval


Generated tags: ['ЦМФ', 'гильдия', 'повышения квалификации', 'предпочтения', 'гибкие программы']
"Мурр... Новиград - город контрастов, да. Но в ЦМФ, как и в других ЗМС, люди превратились в котов, все заняты только собой. Никто не хочет протянуть лапку товарищу, помочь с грузом. Все в напряжении, в спешке. Мурр, раньше было лучше, когда работали втроем, а не порознь. " 



...

Generated tags: ['технолог', 'станция', 'зарплаты', 'мечты', 'Крыса']
Вот, знаете, раньше приходилось порой подолгу ждать, пока заправишся. А сейчас, с этoй предоплатой, всё так гладко!  Заезжаешь, оплачиваешь, и вперёд!  Никаких очередей, никаких задержек. Просто супер! 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    68 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19524.02 ms /    87 tokens
Llama.generate: 186 prefix-match hit, remaining 105 prompt tokens to eval


Generated tags: ['Теги: ожидание', 'заправка', 'предоплата', 'очередь', 'задержка', 'комфорт']
Ну, вот, в Карбоне, например, всё норaльно. Вода чистая, прямо из крана. А вот в глубинке, где ЗМС открываються в новыx меcтах, бывает, приходится жdacь доставки.  Иногда, прав4а, бывает, что... э-э... что вода не такая чиcтая, как хоч4лось бы. Но, в обшему, всё терпимо. 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   105 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   25538.38 ms /   124 tokens


Generated tags: ['Карбоновая вода', 'ЗМС', 'добыча', 'неочевидные трудности', 'терпимость']


In [ ]:
for text in quotes[12:15]:
  print(text)
  prompt = (
          f"Есть ряд тегов, перечисленных через запятую: {', '.join(unique_tags)}"
          "Далее будет дан текст. Выбери 5-7 из данного списка тегов только те, которые точно характеризуют данный текст. Смотри на ключевые слова или их синонимы. Но таких тегов может и не быть\
          Теги: Новиград, ЦМФ, ЗМС, эгоизм, раньше было лучше, Работа"
          f"Текст: {text}\n Теги:"
      )

  tags = generate_tags(prompt, llm)
  print("Generated tags:", tags)

«У нас каждая смена, как маленькая семья. Естественно, все взрослые люди, все с разным темпераментом и характером, но чтобы они какие-то были критические, которые нужно было решать, вплоть до перехода из этой смены в другую, потому что не нашли общего языка, нет, у меня таких вопросов не было»


Llama.generate: 236 prefix-match hit, remaining 98 prompt tokens to eval
llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    98 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    31 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   31999.95 ms /   129 tokens
Llama.generate: 261 prefix-match hit, remaining 68 prompt tokens to eval


Generated tags: ['ЗМС', 'эгоизм', 'раньше было лучше', 'РаботаТекст', 'Зренья', 'ЦМФ', 'Развитие', 'Крыса', 'Мурр.']
Хотчу скать, шо работа в ЦМФ - ето просто супеRRR! Зарплата, конечно, выше чем в "Скиррисе", а скидки на масле в наших маслостанциях просто бомба!  💣 123  



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    68 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    33 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   28880.37 ms /   101 tokens
Llama.generate: 261 prefix-match hit, remaining 125 prompt tokens to eval


Generated tags: ['Теги: ЦМФ', 'Зарплата', 'Скидки', 'Маслостанция', 'РаботаТекст', 'Эгоизм', 'Раньше было лучше']
В ЦМФ, так ска3ть, зарплата вроде и не малaя, но вот возможности для роста, так ска3ть, ограничены. Я, так ска3ть, ужe полгода как технолог, но всё эшe на том же уровне, так ска3ть, как и пришел. В "Коимбрах Масел", так ска3ть, говорЯт, что продвижение по карьерной лестнице там, так ска3ть, более динамичное. Вот и думaю, так ска3ть, стоит ли менять работу. 





llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   125 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    18 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   27384.63 ms /   143 tokens


Generated tags: ['ЦМФ', 'ЗМС', 'эгоизм', 'раньше было лучше', 'повышение квалификации', 'развитие', 'сотрудники']
